In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df1=pd.read_csv('../input/temporary-outputs/xgb_clf_predictions.csv')
df2=pd.read_csv('../input/temporary-outputs/lgbm_clf_predictions.csv')

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df3=df1.merge(df2,on='id')

In [ ]:
df3.head()

In [ ]:
len(df3[df3.target_x!=df3.target_y])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
lr=LogisticRegression()
train_df=df3[df3.kfold_x!=4]
test_df=df3[df3.kfold_x==4]

x_train=train_df[['xgbclf_pred','lgbmclf_pred']]
x_test=test_df[['xgbclf_pred','lgbmclf_pred']]
y_train=train_df.target_x
y_test=test_df.target_x

In [ ]:
x_train

In [ ]:
import sklearn
import joblib

In [ ]:
lr.fit(x_train,y_train)
y_pred=lr.predict_proba(x_test)[:,1]
roc=sklearn.metrics.roc_auc_score(y_test,y_pred)
print(roc)

In [ ]:
df_test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
df_test = df_test.drop(columns = 'id')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
model_0_lgbm= joblib.load('../input/temporary-outputs/model_lgbmclf0 (1)')
model_1_lgbm=joblib.load('../input/temporary-outputs/model_lgbmclf1 (1)')
model_2_lgbm= joblib.load('../input/temporary-outputs/model_lgbmclf2 (1)')
model_3_lgbm= joblib.load('../input/temporary-outputs/model_lgbmclf3 (2)')
model_4_lgbm= joblib.load('../input/temporary-outputs/model_lgbmclf4 (1)')

In [ ]:
y_final_32 = model_3_lgbm.predict_proba(df_test)[:,1]
y_final_02 = model_0_lgbm.predict_proba(df_test)[:,1]
y_final_12 = model_1_lgbm.predict_proba(df_test)[:,1]
y_final_22 = model_2_lgbm.predict_proba(df_test)[:,1]
y_final_42 = model_4_lgbm.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg2 = (y_final_02 + y_final_12 +y_final_22 + y_final_32 + y_final_42)/5

In [ ]:
y_final_avg2

In [ ]:
model_0_xgb= joblib.load('../input/temporary-outputs/model_xgbclf0 (1)')
model_1_xgb =joblib.load('../input/temporary-outputs/model_xgbclf1 (2)')
model_2_xgb= joblib.load('../input/temporary-outputs/model_xgbclf2 (1)')
model_3_xgb= joblib.load('../input/temporary-outputs/model_xgbclf3 (2)')
model_4_xgb= joblib.load('../input/temporary-outputs/model_xgbclf4 (2)')

In [ ]:
y_final_31 = model_3_xgb.predict_proba(df_test)[:,1]
y_final_01 = model_0_xgb.predict_proba(df_test)[:,1]
y_final_11 = model_1_xgb.predict_proba(df_test)[:,1]
y_final_21 = model_2_xgb.predict_proba(df_test)[:,1]
y_final_41 = model_4_xgb.predict_proba(df_test)[:,1]

In [ ]:
y_final_avg1 = (y_final_01 + y_final_11 +y_final_21 + y_final_31 + y_final_41)/5

In [ ]:
y_final_avg1

In [ ]:
d={'xgbclf_pred':y_final_avg1,'lgbmclf_pred':y_final_avg2}
df4=pd.DataFrame(d)

In [ ]:
df4.head()

In [ ]:
y_pred1=lr.predict_proba(df4)[:,1]
submission['target'] = y_pred1

In [ ]:
submission['target']=submission['target'].rank()

In [ ]:
submission['target']

In [ ]:
submission.to_csv('stacked_xgboost_lgbm2.csv',index = False)